# Aim:-  To Build a classification/regression model to predict scores for any new reviewer on Trip_Advisor_Reviews_LasVegas dataset. 

###  loading usefull tools

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


### loading dataset and exploratory data analysis

 Dataset contains categorical feature, we need to deal with it.. we will deal with this later!!

In [2]:
data=pd.read_csv("lasvegas_tripadvisor.csv")
print('Number of data points : ', data.shape[0])
print('Number of features : ', data.shape[1])
print('Features : ', data.columns)
print(data.shape)
data.head(5)

Number of data points :  504
Number of features :  20
Features :  Index(['User country', 'Nr. reviews', 'Nr. hotel reviews', 'Helpful votes',
       'Score', 'Period of stay', 'Traveler type', 'Pool', 'Gym',
       'Tennis court', 'Spa', 'Casino', 'Free internet', 'Hotel name',
       'Hotel stars', 'Nr. rooms', 'User continent', 'Member years',
       'Review month', 'Review weekday'],
      dtype='object')
(504, 20)


,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


### Looking for missing datapoint if there any and also the description about each feature

Since each faeture below contains 504 data points so there is no missing data points!! 

In [3]:
display(data.describe(include=[np.number]))
display(data.describe(exclude=[np.number]))



,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Hotel stars,Nr. rooms,Member years
count,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000,504.000000
mean,48.130952,16.023810,31.751984,4.123016,4.047619,2196.380952,0.767857
std,74.996426,23.957953,48.520783,1.007302,0.844650,1285.476807,80.692897
min,1.000000,0.000000,0.000000,1.000000,3.000000,188.000000,-1806.000000
25%,12.000000,5.000000,8.000000,4.000000,3.000000,826.000000,2.000000
50%,23.500000,9.000000,16.000000,4.000000,4.000000,2700.000000,4.000000
75%,54.250000,18.000000,35.000000,5.000000,5.000000,3025.000000,6.000000
max,775.000000,263.000000,365.000000,5.000000,5.000000,4027.000000,13.000000


,User country,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,User continent,Review month,Review weekday
count,504,504,504,504,504,504,504,504,504,504,504,504,504
unique,48,4,5,2,2,2,2,2,2,21,6,12,7
top,USA,Mar-May,Couples,YES,YES,NO,YES,YES,YES,Hilton Grand Vacations on the Boulevard,North America,March,Wednesday
freq,217,128,214,480,480,384,384,456,480,24,295,42,85


In [4]:
categorical = list(data.select_dtypes(include=['object']).columns.values)
categorical

['User country',
 'Period of stay',
 'Traveler type',
 'Pool',
 'Gym',
 'Tennis court',
 'Spa',
 'Casino',
 'Free internet',
 'Hotel name',
 'User continent',
 'Review month',
 'Review weekday']

### Feature Engineering: 

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# seasons in place of months for better understanding of data

['Dec-Feb' 'Mar-May' 'Jun-Aug' 'Sep-Nov']
data['Period of stay'] = data['Period of stay'].map({'Dec-Feb':'winter', 'Mar-May':'spring', 'Jun-Aug' :'summer','Sep-Nov':'autumn'})

### Conversion of catogorical data into numerical using label Encoding!!

Similar data inside the same category will be assigned same value say 0 or 1 or anything 
depands on number of different values in that category

In [6]:
for i in range(0, len(categorical)):
    data[categorical[i]] = le.fit_transform(data[categorical[i]])
    
data.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,46,11,4,13,5,3,3,0,1,0,0,1,1,2,3,3773,3,9,4,4
1,46,119,21,75,3,3,0,0,1,0,0,1,1,2,3,3773,3,3,4,0
2,46,36,9,25,5,1,2,0,1,0,0,1,1,2,3,3773,3,2,3,2
3,45,14,7,14,4,1,3,0,1,0,0,1,1,2,3,3773,2,6,3,0
4,3,5,5,2,4,1,4,0,1,0,0,1,1,2,3,3773,3,7,7,5


### Spliting dataset into ttrain and test : 

Randomly we will generate train and test dataset out of original data set!!

In [7]:
from sklearn.model_selection import train_test_split
X= data.drop(['Score'], axis=1) ## remove score label from data
y = data['Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=39)
print("Number of X_train data :",X_train.shape) 

print("Number of y_train data :",y_train.shape)

print("Number of X_test data :",X_test.shape) 

print("Number of y_test data :",y_test.shape) 


Number of X_train data : (403, 19)
Number of y_train data : (403,)
Number of X_test data : (101, 19)
Number of y_test data : (101,)


### Forward feature selection!!

We will use SequentialFeatureSelector from mlxtent to cout importnant features!!

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.classifier import StackingClassifier

clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
sfs1 = sfs(clf,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)
sfs1 = sfs1.fit(X_train, y_train)



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:   46.1s finished

[2019-01-31 12:00:16] Features: 1/5 -- score: 0.4639780224806468[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   44.7s finished

[2019-01-31 12:01:00] Features: 2/5 -- score: 0.4639780224806468[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:   43.3s finished

[2019-01-31 12:01:44] Features: 3/5 -- score: 0.48866857054646495[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:   58.5s finished

[2019-01-31 12:02:43] Features: 4/5 -- score: 0.4861369249768447[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   39.1

In [11]:
X.columns

Index(['User country', 'Nr. reviews', 'Nr. hotel reviews', 'Helpful votes',
       'Period of stay', 'Traveler type', 'Pool', 'Gym', 'Tennis court', 'Spa',
       'Casino', 'Free internet', 'Hotel name', 'Hotel stars', 'Nr. rooms',
       'User continent', 'Member years', 'Review month', 'Review weekday'],
      dtype='object')

### Comment:

Below code gives us the index at which the most importanat feature is present:

In [12]:
feat_cols = list(sfs1.k_feature_idx_)
print("Importanat faetures are present at the index:")
print(feat_cols)

Importanat faetures are present at the index:
[6, 7, 9, 11, 13]


In [13]:
## creating main feature on top of 
main_features = ['Traveler type','Pool','Tennis court','Casino','Hotel name']


In [14]:
train_X = X_train[main_features]# main feature of X_train
train_y=y_train# y_train we already have!
test_X= X_test[main_features] # main feature of X_train
test_y=y_test #is alredy defined!

### Applying Different models Counting accuracy !!

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn import svm
import numpy as np
from sklearn import metrics

clf1 = KNeighborsClassifier(n_neighbors=10)
clf1.fit(train_X,train_y)
prediction=clf1.predict(test_X)
print("Accuracy using KNN :",metrics.accuracy_score(prediction,test_y))

## Random forest 
clf2 = RandomForestClassifier(max_depth=3,random_state=0)
clf2.fit(train_X,train_y)
prediction1=clf2.predict(test_X)
print("Accuracy using RandomForest:",metrics.accuracy_score(prediction1,test_y))

###SVM
clf3=svm.SVC(C=10, kernel='linear')
clf3.fit(train_X,train_y)
prediction2=clf3.predict(test_X)
print("Accuracy using SVM :",metrics.accuracy_score(prediction2,test_y))

## Logistic regression
clf4 = LogisticRegression()
clf4.fit(train_X,train_y)
prediction3=clf4.predict(test_X)
print("Accuracy using Logistic Regression :",metrics.accuracy_score(prediction3,test_y))

## Stacking all together
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=clf4)

print('3-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'SVM'
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, cv=3,
                                               scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))




Accuracy using KNN : 0.4752475247524752
Accuracy using RandomForest: 0.48514851485148514
Accuracy using SVM : 0.49504950495049505
Accuracy using Logistic Regression : 0.48514851485148514
3-fold cross validation:

Accuracy: 0.31 (+/- 0.07) [KNN]
Accuracy: 0.45 (+/- 0.02) [Random Forest]


### Comment:

Above aucuracy is not that great.. accuracy using SVM is max i.e 49.5%
